In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
result = pd.DataFrame(test.Applicant_ID)

In [3]:
train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [4]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [5]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [6]:
train=train.fillna(-999)
train

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,-999.000000,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,-999.0,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.282,-999.0,956940.0,-999.0,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,-999.000000,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,-999.0,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55995,Apcnt_999968,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,...,0.000000,9.09,0.730040,-999.0,0.0,lending,591.048913,0.000000,0.241598,no
55996,Apcnt_999972,3360.0,2.01145,0.6252,0.0000,0.000,-999.0,927765.0,-999.0,-999.0,...,0.297913,30.30,0.498208,-999.0,0.0,lending,-999.000000,0.000000,-999.000000,yes
55997,Apcnt_999980,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,...,0.550000,3.03,0.450512,0.0,0.0,lending,1252.474467,2.000000,0.163153,no
55998,Apcnt_999988,3280.0,0.05235,2.0916,2.2212,0.000,-999.0,3458599.0,-999.0,115533.0,...,0.942854,0.00,0.705048,1.0,0.0,charge,-999.000000,0.000000,0.002997,no


In [7]:
train.set_index(['Applicant_ID'],inplace=True)

In [8]:
train

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,-999.000000,1.129518,0.044335,no
Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,-999.0,0.0,charge,349.805730,1.620483,0.322436,no
Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.282,-999.0,956940.0,-999.0,192944.0,1079864.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,-999.000000,1.513370,0.011640,yes
Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,0.488884,2.02,0.685168,-999.0,0.0,charge,89.940100,0.664452,0.082729,no
Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Apcnt_999968,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,19910965.0,...,0.000000,9.09,0.730040,-999.0,0.0,lending,591.048913,0.000000,0.241598,no
Apcnt_999972,3360.0,2.01145,0.6252,0.0000,0.000,-999.0,927765.0,-999.0,-999.0,1849306.0,...,0.297913,30.30,0.498208,-999.0,0.0,lending,-999.000000,0.000000,-999.000000,yes
Apcnt_999980,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,704090.0,...,0.550000,3.03,0.450512,0.0,0.0,lending,1252.474467,2.000000,0.163153,no


In [9]:
train['default_status']=train.default_status.apply(lambda x:1 if x=='yes' else 0)
train['form_field47']=train.form_field47.apply(lambda y:0 if y=='charge' else 1)

In [10]:
train

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000000,3436.0,0.28505,1.6560,0.0000,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,0,-999.000000,1.129518,0.044335,0
Apcnt_1000004,3456.0,0.67400,0.2342,0.0000,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,-999.0,0.0,0,349.805730,1.620483,0.322436,0
Apcnt_1000008,3276.0,0.53845,3.1510,0.0000,6.282,-999.0,956940.0,-999.0,192944.0,1079864.0,...,0.162965,18.18,0.791136,0.0,0.0,0,-999.000000,1.513370,0.011640,1
Apcnt_1000012,3372.0,0.17005,0.5050,0.0000,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,...,0.488884,2.02,0.685168,-999.0,0.0,0,89.940100,0.664452,0.082729,0
Apcnt_1000016,3370.0,0.77270,1.1010,0.0000,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,...,0.275000,12.12,0.438168,0.0,0.0,0,97.887502,1.427891,0.045630,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Apcnt_999968,3740.0,0.01730,0.0000,0.0000,0.000,770998.0,9637475.0,4047934.0,11641992.0,19910965.0,...,0.000000,9.09,0.730040,-999.0,0.0,1,591.048913,0.000000,0.241598,0
Apcnt_999972,3360.0,2.01145,0.6252,0.0000,0.000,-999.0,927765.0,-999.0,-999.0,1849306.0,...,0.297913,30.30,0.498208,-999.0,0.0,1,-999.000000,0.000000,-999.000000,1
Apcnt_999980,3500.0,0.76640,0.0000,0.0000,0.000,118645.0,3662435.0,3662435.0,3585024.0,704090.0,...,0.550000,3.03,0.450512,0.0,0.0,1,1252.474467,2.000000,0.163153,0


In [11]:
train.corr()

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
form_field1,1.000000,0.066395,0.358717,0.359764,0.355599,0.155828,0.123642,0.230769,0.229502,0.172198,...,0.413058,0.381166,0.579925,0.240394,0.281385,0.090533,0.054497,0.361472,0.040527,-0.140965
form_field2,0.066395,1.000000,0.080174,0.080579,0.080074,-0.138547,-0.032814,-0.090430,-0.026720,-0.027382,...,0.070366,0.080117,0.094005,-0.021014,0.105363,-0.097002,-0.002065,0.080659,0.024883,0.073846
form_field3,0.358717,0.080174,1.000000,0.999254,0.993343,0.025861,0.024715,0.038259,0.041953,0.031477,...,0.491087,0.787299,0.241327,0.067352,0.127518,0.006916,0.010087,0.999499,0.006288,0.009359
form_field4,0.359764,0.080579,0.999254,1.000000,0.991149,0.026420,0.025235,0.039185,0.042613,0.032431,...,0.490614,0.786950,0.242454,0.067842,0.133351,0.007203,0.010339,0.998938,0.006456,0.009983
form_field5,0.355599,0.080074,0.993343,0.991149,1.000000,0.021725,0.020352,0.032050,0.034483,0.025820,...,0.488447,0.780580,0.232281,0.056543,0.121485,0.003183,0.008272,0.992554,0.005129,0.015713
form_field6,0.155828,-0.138547,0.025861,0.026420,0.021725,1.000000,0.207660,0.522087,0.289385,0.258334,...,0.018392,0.009114,0.092688,0.179975,-0.005218,0.139724,0.030454,0.025554,0.067747,-0.173197
form_field7,0.123642,-0.032814,0.024715,0.025235,0.020352,0.207660,1.000000,0.357153,0.293446,0.900846,...,0.045296,0.038585,0.104369,0.235327,0.053012,0.057427,0.073881,0.025518,0.063251,-0.120380
form_field8,0.230769,-0.090430,0.038259,0.039185,0.032050,0.522087,0.357153,1.000000,0.613040,0.472565,...,0.054251,0.053338,0.162012,0.261659,0.027039,0.176958,0.131430,0.040402,0.134541,-0.238216
form_field9,0.229502,-0.026720,0.041953,0.042613,0.034483,0.289385,0.293446,0.613040,1.000000,0.497936,...,0.090142,0.091385,0.164731,0.348559,0.093447,0.176700,0.154381,0.046301,0.224912,-0.228493
form_field10,0.172198,-0.027382,0.031477,0.032431,0.025820,0.258334,0.900846,0.472565,0.497936,1.000000,...,0.066230,0.062939,0.136276,0.319098,0.094964,0.098030,0.112312,0.033265,0.102562,-0.169563


In [12]:
#Prepare the Data set for training the model
target = train['default_status']
train = train.drop(['default_status'], axis=1)

In [13]:
scaler = MinMaxScaler() 
train = scaler.fit_transform(train)
train

array([[9.05286793e-01, 9.82566630e-01, 9.47257575e-01, ...,
        0.00000000e+00, 9.73835947e-01, 4.32940029e-06],
       [9.09369259e-01, 9.82949072e-01, 9.45911647e-01, ...,
        2.53481819e-05, 9.74314005e-01, 4.33060546e-06],
       [8.72627067e-01, 9.82815790e-01, 9.48672797e-01, ...,
        0.00000000e+00, 9.74209708e-01, 4.32925861e-06],
       ...,
       [9.18350684e-01, 9.83039926e-01, 9.45689945e-01, ...,
        4.23120862e-05, 9.74683544e-01, 4.32991520e-06],
       [8.73443560e-01, 9.82337823e-01, 9.47669930e-01, ...,
        0.00000000e+00, 9.72736125e-01, 4.32922115e-06],
       [9.22841396e-01, 9.82747797e-01, 9.45689945e-01, ...,
        1.12479985e-04, 9.75657254e-01, 4.33214737e-06]])

In [14]:
# PREPROCESS THE TEST DATA
test['form_field47']=test.form_field47.apply(lambda y:0 if y=='charge' else 1)
test.set_index(['Applicant_ID'],inplace=True)
test=test.fillna(-999)
test

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0000,418564.0,418564.0,418564.0,540710.0,0.0,...,-999.0000,0.825000,1.01,0.800000,-999.0,0.0,0,-999.000000,0.000000,0.011221
Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0000,0.0,9858816.0,49014.0,1510098.0,18308285.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,1,-999.000000,0.504974,0.043525
Apcnt_1000052,-999.0,0.27505,0.0600,0.0000,0.0000,-999.0,-999.0,-999.0,-999.0,0.0,...,-999.0000,-999.000000,0.00,-999.000000,-999.0,-999.0,0,-999.000000,0.000000,-999.000000
Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0000,0.0,473802.0,473802.0,1724437.0,493641.0,...,-999.0000,0.916663,2.02,0.464224,-999.0,-999.0,0,90.163742,0.788809,0.104029
Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0000,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,...,-999.0000,0.234047,23.23,0.726688,0.0,0.0,1,1303.587148,1.637733,0.163124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Apcnt_999940,3258.0,0.37925,0.5698,5.1282,0.0000,-999.0,2280318.0,-999.0,-999.0,2280318.0,...,92.6910,1.100000,0.00,0.800000,-999.0,1.0,0,-999.000000,0.000000,-999.000000
Apcnt_999956,3514.0,0.95260,0.0000,0.0000,0.0000,1494538.0,4773419.0,4773419.0,89718571.0,12999991.0,...,125.0220,0.000000,12.12,0.196400,0.0,0.0,0,4200.072959,3.000000,0.930931
Apcnt_999976,3302.0,0.05755,4.2394,0.0000,25.4376,-999.0,49403.0,-999.0,-999.0,49403.0,...,-999.0000,1.100000,1.01,0.800000,-999.0,0.0,0,-999.000000,4.855060,-999.000000


In [15]:
test = scaler.fit_transform(test)
test

array([[8.64462135e-01, 9.78492141e-01, 9.76449129e-01, ...,
        0.00000000e+00, 9.76539589e-01, 3.95710610e-06],
       [8.74260053e-01, 9.79397689e-01, 9.69444088e-01, ...,
        0.00000000e+00, 9.77033210e-01, 3.95723406e-06],
       [0.00000000e+00, 9.78419979e-01, 9.66637621e-01, ...,
        0.00000000e+00, 9.76539589e-01, 0.00000000e+00],
       ...,
       [8.77934272e-01, 9.78207018e-01, 9.70681387e-01, ...,
        0.00000000e+00, 9.81285493e-01, 0.00000000e+00],
       [8.84466218e-01, 9.78891577e-01, 9.67975545e-01, ...,
        8.45407021e-06, 9.77339915e-01, 3.95715572e-06],
       [8.82424985e-01, 9.78589711e-01, 9.68821375e-01, ...,
        0.00000000e+00, 9.76539589e-01, 3.95861409e-06]])

In [16]:
rfc= RandomForestClassifier(n_estimators=500)
Catboost= CatBoostClassifier(verbose=False)
LGBM= LGBMClassifier()
Classifiers=[('rfc',rfc),('Catboost',Catboost),('LGBM',LGBM)]
vc=VotingClassifier(estimators=Classifiers,voting='soft')

In [17]:
# StraifiedKFold
splits = 25
skfold = StratifiedKFold(n_splits=splits, random_state=42, shuffle=True)
scores = []
preds = []
i = 1

for train_split, test_split in skfold.split(train,target):
    rf = vc
    x_train, x_test, y_train, y_test = train[train_split], train[test_split], target[train_split], target[test_split]
    rf.fit(x_train, y_train)
    score = roc_auc_score(y_test, rf.predict_proba(x_test)[:,1])   
    pred = rf.predict_proba(test)[:,1]
    scores.append(score)
    preds.append(pred)
    print('Roc score {} split: '.format(i), score)
    i +=1
 
test_pred = np.mean(preds, axis=0)
print('Average Roc Score: ', np.mean(scores))

Roc score 1 split:  0.8698846870632085
Roc score 2 split:  0.8570074286897552
Roc score 3 split:  0.8364577401597902
Roc score 4 split:  0.8393837034736242
Roc score 5 split:  0.8386330693171818
Roc score 6 split:  0.8216704629773428
Roc score 7 split:  0.8311159427792446
Roc score 8 split:  0.8321167883211679
Roc score 9 split:  0.8344204586633535
Roc score 10 split:  0.8398053959379477
Roc score 11 split:  0.8468480404671038
Roc score 12 split:  0.8330850457635464
Roc score 13 split:  0.8330936631195476
Roc score 14 split:  0.8475255800827051
Roc score 15 split:  0.8425759862294651
Roc score 16 split:  0.8350228736943359
Roc score 17 split:  0.8383039319918265
Roc score 18 split:  0.8470893264351399
Roc score 19 split:  0.8583145097963181
Roc score 20 split:  0.8515014127077994
Roc score 21 split:  0.8467801787885936
Roc score 22 split:  0.8289239399844242
Roc score 23 split:  0.8344196587742457
Roc score 24 split:  0.8448649714173073
Roc score 25 split:  0.8414697331527997
Average R

In [18]:
ss=pd.read_csv("SampleSubmission.csv")
ss["default_status"]=test_pred
ss.to_csv("MySubmision.csv",index=False)